In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import tqdm
import json

from codebook.python import *
from codebook.semantics import DSNotebooks as DSN

Evaluator.use_ds_gh_2019()
Evaluator.create_sqlite_db()

In [2]:
DSN.pandas_reads()

  + Query already compiled (cached) `/tmp/queries/1aa4a0ef2179e9261ba80b7b5cd1d82f711654ac52c104cf10c25d1a9c2878aa.dl`
  - TODO: re-enable writing pre-filters.
  + File select time: 0.0000s
  + Query time: 71.8299s
  + Collation time: 0.0126s
Total time: 71.9286s
  + Profile time: 0.5941s
  + Compile time: 15.3591s
  + Had only 87 allowable files (pre-filter files)
  - TODO: re-enable writing pre-filters.
  + File select time: 0.0001s
  + Query time: 191.3716s
  + Collation time: 0.0077s
Total time: 207.4347s
  + Profile time: 0.6896s
  + Compile time: 17.0051s
  + Had only 0 allowable files (pre-filter files)
  - TODO: re-enable writing pre-filters.
  + File select time: 0.0006s
  + Query time: 185.0140s


ValueError: Length mismatch: Expected axis has 7 elements, new values have 8 elements

In [7]:

import sqlite3
con = sqlite3.connect('/tmp/query-outputs/results.db')
pd.read_sql('SELECT * FROM synth_query', con)

,0,1,2,3,4,5,6,7
0,-8951724140759883532,keras,from keras import optimizers,159,0,159,28,-3921269869785305315
1,-8951724140759883532,keras,from keras import layers,158,0,158,24,5236203477491735502
2,-8951724140759883532,keras,from keras import models,157,0,157,24,6415170120986042966
3,-8951724140759883532,shutil,from shutil import copyfile,10,0,10,27,5418350819378887433
4,-8951724140759883532,keras.preprocessing.image,from keras.preprocessing.image import ImageDat...,189,0,189,56,6699661111957688440
...,...,...,...,...,...,...,...,...
1402,8749042170917698348,keras.layers.core,"from keras.layers.core import Dense, Dropout, ...",16,0,16,56,-1623305689753394692
1403,8749042170917698348,sklearn.svm,from sklearn.svm import SVC,12,0,12,27,-4634181180916259667
1404,8888034084107860641,numpy,import numpy as np,6,0,6,18,1329089894859587542
1405,8888034084107860641,sklearn,"from sklearn import cross_validation, datasets...",4,0,4,69,7470684533828658014


In [40]:
from codebook.python import *

# Code "shape" stuff
counts = {}
counts['num_functions'] = len(execute(function_def() % 'func'))
counts['num_classes'] = len(execute(class_def() % 'class'))
counts['num_ifs'] = len(execute(if_statement() % 'if'))
counts['num_for_loops'] = len(execute(for_loop() % 'loop'))
counts['num_while_loops'] = len(execute(while_loop() % 'loop'))

# Nesting
counts['num_func_in_class'] = len(execute(
  function_def() % 'func'
    |where| any_parent()
      |isa| class_def()
))
counts['num_func_in_func'] = len(execute(
  function_def() % 'func'
    |where| any_parent()
      |isa| function_def()
))
counts['num_class_in_func'] = len(execute(
  class_def() % 'class'
    |where| any_parent()
      |isa| function_def()
))
counts['num_class_in_class'] = len(execute(
  class_def() % 'class'
    |where| any_parent()
      |isa| class_def()
))

counts

  + File select time: 0.0015s


In [2]:
from codebook.python import *

# Most popular imports (either import * or from * import *)
query = ( imports() % select('module_name') % 'import' )

# Get results
imports_df = execute(query)
imports_with_counts = imports_df.value_counts(subset=['out_module_name_import'])

# Get number of unique libraries imported
unique_libraries = imports_df.out_module_name_import.nunique()
print("Unique libraries: {}".format(unique_libraries))

# Show the imports by count table
visualize(imports_df, 'import')

  + File select time: 0.0008s
  + Query time: 5.9261s
  + Collation time: 0.0426s
Total time: 5.9712s
Unique libraries: 1691


In [4]:
imports_with_counts.head(10)

out_module_name_import
numpy                     443
matplotlib.pyplot         300
pandas                    261
os                        132
math                      116
sklearn                   111
matplotlib                 99
IPython.display            84
seaborn                    72
time                       66
dtype: int64

In [5]:
from codebook.python import *

# Number of regular imports vs number of from ... imports
num_regular_imports = len(execute(
  import_stmt() % 'import'
))
                          
num_from_imports = len(execute(
  import_from_stmt() % 'import_from'
))

percent = float(num_from_imports)/float(num_regular_imports)

print("Regular imports: {}".format(num_regular_imports))
print(" + from imports: {} ({:.2%})".format(num_from_imports, percent))

  + File select time: 0.0005s
  + Query time: 4.6194s
  + Collation time: 0.0207s
Total time: 4.6429s
  + File select time: 0.0005s
  + Query time: 4.6329s
  + Collation time: 0.0190s
Total time: 4.6541s
Regular imports: 2651
 + from imports: 1769 (66.73%)


In [46]:
query = (
  call(with_name('__cb_magic')) % 'call'
)

print("Looking for cell magic:")
visualize(execute(query, compile=False), 'call')

Looking for cell magic:
  + File select time: 0.4126s
  + Query time: 2.9067s
  + Collation time: 0.0095s
Total time: 3.3343s


In [1]:
from codebook.python import *

selected_imports = (
  # Matches `import pandas...` and 
  # `from pandas.. import ...` statements
  imports() 
  |where| the_module_root() 
    |is_| anything(with_text('pandas'))
)

pd_query = lambda specific=None: (
  # Matches read_csv calls where the target
  # is coming from a pandas.. import
  (call(with_name(specific)) % 'call' 
   if specific else call() % select('name') % 'call')
  |where| call_target()
    |isa| use_of(selected_imports) % 'use'
)

# df_calls_to_read_csv = execute(pd_query('read_csv'), compile=True)
# df_calls_from_pandas = execute(pd_query(), compile=True)

In [8]:
print('Top 10 most used pandas.* APIs:')
df_calls_from_pandas.out_name_call.value_counts().head(10)

Top 10 most used pandas.* APIs:


DataFrame      317
read_csv       242
Series          73
concat          71
merge           44
read_excel      31
set_option      24
get_dummies     20
read_table      19
to_datetime     18
Name: out_name_call, dtype: int64

In [10]:
visualize(df_calls_to_read_csv, 'call')

In [1]:
from codebook.python import *

selected_imports = (
  imports() 
  |where| the_module_root() 
    |is_| anything(with_text('pandas'))
)

read_csv_calls = (
  call(with_name('read_csv')) % 'read'
  |where| call_target()
    |isa| use_of(selected_imports)
)

fit_calls = (
  call(with_name('fit')) % 'fit'
)

sources = execute(read_csv_calls, compile=True)
sinks = execute(fit_calls, compile=True)

  + File select time: 0.7965s
  + Compile time: 43.2836s
  + Query time: 1.6357s
  + Collation time: 0.0087s
Total time: 44.9302s
  + File select time: 0.4950s
  + Compile time: 21.1431s
  + Query time: 1.2334s
  + Collation time: 0.0062s
Total time: 22.3840s


In [8]:
from codebook.python import *

selected_imports = (
  # Matches `import pandas...` and 
  # `from pandas.. import ...` statements
  imports() 
  |where| the_module_root() 
    |is_| anything(with_text('pandas'))
)

pandas_calls = execute(
  # Matches any call where the target
  # is coming from a pandas.. import
  call() % select('name') % 'call'
  |where| call_target()
    |isa| use_of(selected_imports) % 'use'
)

pandas_calls.out_name_call.value_counts().head(10)

  + File select time: 0.5178s
  + Query time: 22.4577s
  + Collation time: 0.0148s
Total time: 22.9919s


DataFrame      317
read_csv       242
Series          73
concat          71
merge           44
read_excel      31
set_option      24
get_dummies     20
read_table      19
to_datetime     18
Name: out_name_call, dtype: int64

In [10]:
visualize(pandas_calls, 'call')